# React `useMemo` Hook 

 

***

## Introduction  

- This lecture explains the **React `useMemo` hook**, when to use it, and how it helps in **performance optimization** by avoiding unnecessary expensive calculations. 
- The instructor first builds intuition with the concept of **memoization**, then demonstrates `useMemo` using a React counter app with an artificially expensive calculation. 

***

## React Hooks and the `use` Prefix  

- In React, hooks such as `useState`, `useEffect`, `useForm`, `useNavigate`, and `useDispatch` follow a common naming convention: hook names start with the **`use` prefix**. 
- `useMemo` follows the same convention, indicating that it is also a React hook that plugs into React’s rendering lifecycle. 

***

## Memoization – Core Idea  

### What is Memoization?  

- **Memoization** is a technique where the result of a computation is **stored** so that when the same input appears again, the stored answer is reused instead of recomputing. 
- It is widely used in **dynamic programming (DP)** and recursive solutions in DSA to avoid solving the same subproblem multiple times. 

### Intuitive Example (Student–Teacher)  

- A teacher asks a student:  
  - What is \(2 \times 3\)? The student computes 6 once and stores/remembers it. 
  - Later, when \(2 \times 3\) is asked again, the student returns 6 directly without recalculation. 
- For new questions like \(4 \times 3\) or \(6 \times 6\), the student computes once, stores the result, and then reuses it if asked again. 
- Key idea:  
  - If a problem has **not** been solved before: compute and store the result.  
  - If it **has** been solved before: **do not compute again**; just return the stored result. 

***

## Why Memoization is Useful  

- Recomputing the same result repeatedly wastes **time** and **resources**, especially for heavy tasks like large loops, complex calculations, or expensive API/data operations. 
- Memoization ensures that **already-solved problems are not solved again**, which improves performance in both algorithms (DP) and UI frameworks like React. 

***

## Expensive Operations in React  

### What is an “Expensive” Operation?  

- An **expensive operation** is any task that consumes **a lot of time**, **memory**, or **system resources**. 
- Examples in a React app:  
  - Heavy loops over large ranges.  
  - Complex mathematical computations.  
  - Costly data transformations. 

### Artificial Expensive Task Example  

- The instructor defines an `expensiveTask` function that:  
  - Takes a number as input and returns its double. 
  - To make it expensive, a loop runs from 0 to a very large number (e.g., up to 100 crore) before returning the result. 
- Although doubling itself is simple, the big loop makes the function **slow** on each call. 

***

## React Counter App Setup  

### Basic Counter Component  

- The instructor creates a simple React app (using Vite) with:  
  - A **state variable** `count` and `setCount` managed by `useState`. 
  - A button that increments `count` on each click. 
- Initially, the app runs fast because there are no heavy computations inside the component. 

### Adding the Expensive Task  

- A variable `doubleValue` is introduced:  
  - `let doubleValue = expensiveTask(4);` (first version: hardcoded `4`). 
  - `doubleValue` is rendered in JSX to display the doubled value. 
- After this change, clicking the increment button becomes **noticeably slow**, because `expensiveTask` runs on every render. 

***

## How React Re-Rendering Causes Slowness  

### Render Flow Explanation  

- On button click:  
  - `setCount(count + 1)` is called, which updates the `count` state. 
  - Any state update triggers a **re-render** of the component. 
- On each render, **the entire component function runs again**, including:  
  - The line where `expensiveTask` is called.  
  - The heavy loop inside `expensiveTask`. 

### Observed Behavior  

- Every click on the counter button triggers the expensive loop again, even though the input (like `4`) and thus the result are exactly the same. 
- This repeated unnecessary execution causes the UI to lag and respond slowly to user actions. 

***

## Problem Statement in React Terms  

- The **core problem**: an **expensive calculation is being recalculated on every render** even when its input has not changed. 
- In the example, `expensiveTask` always receives the same value (e.g., `4`), so recomputing the result on every render is wasteful. 
- Desired behavior:  
  - Compute once for a given input.  
  - Reuse the same result until the input changes. 

***

## Introduction to `useMemo`  

### What `useMemo` Does  

- `useMemo` is a React hook used to **memoize** the result of a function so that the function is **only re-run when its dependencies change**. 
- In simpler words:  
  - If the input (dependency) is the same as the last render, `useMemo` returns the **cached value** without calling the function again.  
  - If the input changes, `useMemo` calls the function, stores the new result, and returns it. 

### Basic Signature  

- Conceptually, `useMemo` looks like:  

  ```jsx
  const memoizedValue = useMemo(() => {
    // some calculation using dependencies
    return computeSomething(deps);
  }, [deps]);
  ```


- Parameters:  
  - First argument: a **function** that returns the computed value (the “expensive” calculation). 
  - Second argument: a **dependency array** that tells React when to recompute. 

***

## Applying `useMemo` to the Example  

### Adding Input State  

- Instead of always doubling 4, the instructor adds an **input field** so the user can type a number: 
  - New state: `const [input, setInput] = useState(0);`  
  - The input field is bound to this state using `value={input}` and `onChange={(e) => setInput(e.target.value)}`. 
- The expensive calculation is then changed to use `input` instead of a hardcoded number:  
  - `doubleValue = expensiveTask(input);` 

### Issue Without `useMemo`  

- Even after tying the double calculation to `input`, the problem remains:  
  - Clicking the **counter** button (which does not affect `input`) still causes `expensiveTask(input)` to run on every render. 
- Result: the counter continues to lag, because the expensive function is re-run even when `input` has not changed. 

### Wrapping with `useMemo`  

- The expression is updated to use `useMemo`:  

  ```jsx
  const doubleValue = useMemo(() => {
    return expensiveTask(input);
  }, [input]);
  ```


- Interpretation:  
  - The function passed to `useMemo` computes `expensiveTask(input)`.  
  - The dependency array `[input]` tells React:  
    - Only recompute `doubleValue` when `input` changes.  
    - If `input` stays the same across renders, reuse the previously stored result. 

***

## Behavior After Using `useMemo`  

### When `input` Does Not Change  

- Clicking the counter button updates `count`, causing a re-render, but `input` remains the same. 
- Because the dependency `[input]` has not changed:  
  - `useMemo` does **not** re-run `expensiveTask`.  
  - It returns the **memoized value** from the previous render. 
- Effect:  
  - The counter increments smoothly.  
  - UI is responsive, and lag from the expensive loop is removed when it is unnecessary. 

### When `input` Changes  

- When the user types a new number in the input field:  
  - `setInput` is called, updating `input` and triggering a re-render. 
  - Because `input` in the dependency array has changed, `useMemo` calls `expensiveTask(input)` again to compute the new double. 
- This recalculation is **necessary**, as the output genuinely depends on the new input. 

***

## Important Details and Clarifications  

### Only Last Value Is Cached  

- In this `useMemo` example, React **does not maintain a map for all past inputs**. 
- `useMemo` stores only the **last computed value** associated with the last dependency snapshot. 
- Example:  
  - If you compute for input 3, then for 4, then for 5:  
    - The memoized value corresponds only to the latest input (5).  
    - If you go back to 3 or 4, the function will compute again. 

### `useMemo` vs Algorithmic Memoization  

- In algorithmic memoization (like DP), you often store results for many subproblems in a table or dictionary. 
- In React `useMemo`, you get **one memoized value per hook usage**, tied to the **last dependency values**, not a full history cache. 

### When `useMemo` Does Not Help  

- Changing `input` will always require recomputing the double, because the desired result genuinely depends on it. 
- `useMemo` is not about avoiding **necessary** work; it is about avoiding **unnecessary repetition** when dependencies do not change. 

***

## When to Use `useMemo`  

### Good Use Cases  

- Use `useMemo` when:  
  - You have a **heavy computation** inside a component that runs on every render. 
  - The result of that computation depends on values that often **do not change** across renders. 
  - Recomputing it is visibly slowing down the UI or wasting resources. 

### When to Avoid Overuse  

- Do **not** use `useMemo` everywhere:  
  - Wrapping trivial or cheap calculations in `useMemo` adds complexity without real benefit. 
  - Overusing it can make code harder to read and maintain. 
- A good heuristic:  
  - If you feel “this function is expensive and is being called unnecessarily when inputs haven’t changed”, then `useMemo` is appropriate. 

***

## Reference: Typical `useMemo` Pattern  

```jsx
import { useState, useMemo } from "react";

function Example() {
  const [count, setCount] = useState(0);
  const [input, setInput] = useState(0);

  function expensiveTask(num) {
    // Heavy loop (dummy) to simulate expensive work
    for (let i = 0; i <= 1_000_000_000; i++) {}
    return num * 2;
  }

  const doubleValue = useMemo(() => {
    return expensiveTask(input);
  }, [input]);

  return (
    <>
      <button onClick={() => setCount(count + 1)}>Increment</button>
      <div>Count: {count}</div>

      <input
        type="number"
        value={input}
        onChange={(e) => setInput(e.target.value)}
        placeholder="Enter number"
      />
      <div>Double: {doubleValue}</div>
    </>
  );
}
```

- This pattern mirrors the lecture’s example: an expensive calculation is memoized and only recomputed when `input` changes, not when unrelated state (`count`) updates. 

***

## Summary – Main Takeaways  

- **Memoization** means storing computed results so they can be reused later for the same inputs instead of recomputing. 
- React’s **`useMemo` hook** memoizes the result of a function and recomputes it only when specified dependencies change, helping avoid unnecessary expensive work on re-renders. 
- In the lecture example, an artificially expensive `expensiveTask` made the counter lag, and wrapping it in `useMemo` with `[input]` eliminated useless recalculations when only `count` changed. 
- Use `useMemo` **selectively** wherever there are **unnecessary repeated expensive calculations**, but avoid overusing it for simple or cheap operations. 
